Radiation Damping. Light Scattering

https://www.feynmanlectures.caltech.edu/I_32.html

total amount of light scattered in all directions


In [1]:
using DynamicPolynomials
using JuMP
using MosekTools
using LinearAlgebra
using CSV
using DataFrames
import JSON
using Dates
using Gurobi

functions for generating monomials

In [2]:
function all_monomials_up_to_max_deg(x, deg)
    if size(x,1) == 0
        [1]
    else
    [ x[1]^k * m for k=0:deg 
      for m=all_monomials_up_to_max_deg(x[2:end], deg)
    ]
    end
end

function mons_of_max_degree_and_unit(x, deg, u)
    [m
        for m=all_monomials_up_to_max_deg(x, deg)
        #if all(unit(m) .== u)
    ]
end
                
function degree_poly(p)
    maximum(degree.(monomials(p)))
end

function all_monomials_up_to_max_deg_overall(x, deg, deg_overall)
    if size(x,1) == 0
        [1]
    else
    [ x[1]^k * m for k=0:min(deg, deg_overall) 
                for m=all_monomials_up_to_max_deg_overall(x[2:end], deg, deg_overall-k)
    ]
    end
end

function mons_of_max_degree_and_unit_overall(x, deg, deg_overall, u)
    [m
        for m=all_monomials_up_to_max_deg_overall(x, deg, deg_overall)
        #if all(unit(m) .== u)
    ]
end

function all_monomials_up_to_max_deg_overall_and_individual(x, deg, deg_overall, theDegrees)
    if size(x,1) == 0
        [1]
    else
    [ x[1]^k * m for k=0:min(deg, deg_overall, theDegrees[1]) 
                for m=all_monomials_up_to_max_deg_overall_and_individual(x[2:end], deg, deg_overall-k, theDegrees[2:end])
    ]
    end
end

all_monomials_up_to_max_deg_overall_and_individual (generic function with 1 method)

In [17]:
@polyvar  P S  r  qc ap sin_theta dA dtheta  w x_0 # Intg c eps_0 W Q E  P  q_e x_0 m m_e  r_0  a b lambda  w e c    # the order matters per evaluation # sin_theta, a = sin_theta, b = cos_theta 

x =[  P S  r  qc ap sin_theta dA dtheta w x_0]  # Intg sin_theta 
# w_0 natural fequency of atom vibration

1×10 Matrix{Variable{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}}}:
 P  S  r  qc  ap  sin_theta  dA  dtheta  w  x_0

In [18]:
axioms= 
[
  # a^2 + b^2 - 1,  # a = sin_theta, b = cos_theta     
  # S - eps_0 * c * E^2, #need an expectation over E (I.32.1)
  # S * 16 * pi^2 *eps_0 * r^2 * c^3 - qc^2 * ap^2 * a^2, #sin_theta^2, # rate of radiation of energy (I.32.2)
  S * r^2 - qc^2 * ap^2 * sin_theta^2, ##S * 16 * pi^2 *eps_0 * r^2 * c^3 - qc^2 * ap^2 * sin_theta^2, #sin_theta^2, # rate of radiation of energy (I.32.2)
    dA - 2* pi* r^2 * sin_theta * dtheta, # The area of a spherical segment is 2πrsinθ⋅rdθ
 #  Intg * sin_theta^3 * dtheta - 4/3,            # a crocked way of defining an integral of sin^3 theta between 0 and pi
 #  P - Intg * S * dA,
    P - S * dA, 
    #Intg - sin_theta^3 * dtheta,
    #Intg - 4/3,
    4/3 -  sin_theta^3 * dtheta,
    #P * 6 * pi * eps_0 * c^3 - q^2 * ap^2,                #               (I.32.5)
   ap^2 - 0.5 *w^4 * x_0^2,                              #               (I.32.5b) The average of the acceleration of an ocilating charge  over a cycle (squared)
    #P * 12 * pi * eps_0 * c^3 - q^2 * w^4 * x_0^2,     #               (I.32.6)
 #   e^2 * 4 * pi * eps_0 - q_e^2,                       # electric potenial of an electron (I.32.7)
 #   W - 1/2 * m * w^2 * x_0^2,                          #  total energy of an oscillator, on the average half is kinetic and half is potential energy (I.32.9)
 #   r_0 * m_e * c^2 - e^2,                              # radius magnitude
 #   Q * 4 * pi * r_0 - 3 * lambda,                      # total energy content of the oscillator at any time divided by the energy loss per radian
 #   lambda * w - 2* pi * c,                             # wavelength to angualr frequency relation
    ]

5-element Vector{Polynomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}, Float64}}:
 Sr² - qc²ap²sin_theta²
 dA - 6.283185307179586r²sin_thetadtheta
 P - SdA
 1.3333333333333333 - sin_theta³dtheta
 ap² - 0.5w⁴x_0²

In [20]:
deg = 4
deg_overall = 10
theDegrees = [ 2  2 4 2  2   3         2   2   4   2] # 4 2 ]                                                # [2 2 2 2 2 2  2   2   2 2  2  3  3    2    2 2 2 2 2  2   ] 
            #  P S r qc ap sin_theta dA dtheta  w x_0] # Intg  W Q E S P q q_e x_0 ap m m_e r r_0 lambda a b w e c eps_0             
#theDegrees = [2     2  2   4 3  4  4  3         2  2     4  2]                                                # [2 2 2 2 2 2  2   2   2 2  2  3  3    2    2 2 2 2 2  2   ] 
            # Intg S eps_0 r c qc ap sin_theta dA dtheta w x_0 # W Q E S P q q_e x_0 ap m m_e r r_0 lambda a b w e c eps_0 

candidate_mons = [
    #mons_of_max_degree_and_unit_overall(params, deg, deg_overall, [])
    all_monomials_up_to_max_deg_overall_and_individual(x, deg, deg_overall, theDegrees)
    for ai=axioms
]
@show size.(candidate_mons)

# model = Model(Mosek.Optimizer)
model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "TimeLimit", 600.0)

mons_q = mons_of_max_degree_and_unit_overall([ P qc r w x_0 ], deg, deg_overall, []) #  only include variables expected to appear in the final theorem 
coeff_q =   @variable(model, [1:size(mons_q,1)], base_name="q")

q = sum(ci .* mi for (ci, mi)=zip(coeff_q, mons_q)) # Zip pairs things without needing a ref index, e.g., zip([1, 2, 3], [4,5,6])=((1,4), (2,5), (3,6))
coeff_αs = [
    @variable(model, [1:size(X,1)], base_name="α$i")
    for (i,X)=enumerate(candidate_mons)
    ]
@show size.(coeff_αs)
αs = [sum(ci .* mi) for (ci, mi)=zip(coeff_αs, candidate_mons)]

residual = q - sum(αᵢ * aᵢ for (αᵢ, aᵢ)=zip(αs,axioms));
eqs = coefficients(residual)

# Ensure that the sum of the coefficients on the terms involving m isn't zero, in order that m is part of expression
@constraint model sum(coeff_q[degree.(mons_q, [P ]).>0]) == 4.0 # insist that some variables will appear
#@constraint model sum(coeff_q[degree.(mons_q,[E_c]).>0]) == 0.0 # insist that some variables will not appear
@constraint model eqs .== 0
@objective model Max 0

optimize!(model)
@show termination_status(model)

size.(candidate_mons) = [(58170,), (58170,), (58170,), (58170,), (58170,)]
Set parameter Username
Academic license - for non-commercial use only - expires 2024-12-04
Set parameter TimeLimit to value 600
size.(coeff_αs) = [(58170,), (58170,), (58170,), (58170,), (58170,)]
Set parameter TimeLimit to value 600
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i9-10885H CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 386146 rows, 292603 columns and 584703 nonzeros
Model fingerprint: 0x0ffd25c9
Coefficient statistics:
  Matrix range     [5e-01, 6e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 4e+00]
Presolve removed 386146 rows and 292603 columns
Presolve time: 0.22s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0  

OPTIMAL::TerminationStatusCode = 1

In [1]:
value_poly = p -> sum(value.(coefficients(p)).* monomials(p))


#3 (generic function with 1 method)

In [22]:
value_q = value_poly(q)
value_q

4.0P - 16.755160819145562qc²w⁴x_0²

dividing by 4, we get as expected  P = 4/3*pi q_c^2 x_0^2 w^4

In [23]:
@show value_αs = value_poly.(αs)

value_αs = value_poly.(αs) = Polynomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}, Float64}[25.132741228718345sin_thetadtheta, 4.0S, 4.0, -12.566370614359172qc²w⁴x_0², 25.132741228718345qc²sin_theta³dtheta]


5-element Vector{Polynomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}, Float64}}:
 25.132741228718345sin_thetadtheta
 4.0S
 4.0
 -12.566370614359172qc²w⁴x_0²
 25.132741228718345qc²sin_theta³dtheta

______________________________alternative (squares ) formulation _____________________

In [5]:
@polyvar  P S  rs  qcs aps sin_theta dA dtheta  ws x_0s # Intg c eps_0 W Q E  P  q_e x_0 m m_e  r_0  a b lambda  w e c    # the order matters per evaluation # sin_theta, a = sin_theta, b = cos_theta 

x =[  P S  rs  qcs aps sin_theta dA dtheta  ws x_0s]  # Intg sin_theta 
# w_0 natural fequency of atom vibration

1×10 Matrix{Variable{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}}}:
 P  S  rs  qcs  aps  sin_theta  dA  dtheta  ws  x_0s

In [6]:
axioms= 
[
  # a^2 + b^2 - 1,  # a = sin_theta, b = cos_theta     
  # S - eps_0 * c * E^2, #need an expectation over E (I.32.1)
  # S * 16 * pi^2 *eps_0 * r^2 * c^3 - qc^2 * ap^2 * a^2, #sin_theta^2, # rate of radiation of energy (I.32.2)
  S * rs - qcs * aps * sin_theta^2, ##S * 16 * pi^2 *eps_0 * r^2 * c^3 - qc^2 * ap^2 * sin_theta^2, #sin_theta^2, # rate of radiation of energy (I.32.2)
    dA - 2* pi* rs * sin_theta * dtheta, # 
    sin_theta^3 * dtheta - 4/3,            # a crocked way of defining an integral of sin^3 theta between 0 and pi
    P - S * dA,
    #P * 6 * pi * eps_0 * c^3 - q^2 * ap^2,              #               (I.32.5)
   aps - 0.5 *ws^2 * x_0s,                               #               (I.32.5b)
    #P * 12 * pi * eps_0 * c^3 - q^2 * w^4 * x_0^2,      #               (I.32.6)
 #   e^2 * 4 * pi * eps_0 - q_e^2,                       # electric potenial of an electron (I.32.7)
 #   W - 1/2 * m * w^2 * x_0^2,                          #  total energy of an oscillator, on the average half is kinetic and half is potential energy (I.32.9)
 #   r_0 * m_e * c^2 - e^2,                              # radius magnitude
 #   Q * 4 * pi * r_0 - 3 * lambda,                      # total energy content of the oscillator at any time divided by the energy loss per radian
 #   lambda * w - 2* pi * c,                             # wavelength to angualr frequency relation
    ]

5-element Vector{Polynomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}, Float64}}:
 Srs - qcsapssin_theta²
 dA - 6.283185307179586rssin_thetadtheta
 -1.3333333333333333 + sin_theta³dtheta
 P - SdA
 aps - 0.5ws²x_0s

In [10]:
deg = 3
deg_overall = 12
theDegrees = [1 1 2   2   2   3         1   2      2  2] # 4 2 ]                                                # [2 2 2 2 2 2  2   2   2 2  2  3  3    2    2 2 2 2 2  2   ] 
            # P S rs qcs aps sin_theta dA dtheta  ws x_0s] # W Q E S P q q_e x_0 ap m m_e r r_0 lambda a b w e c eps_0             
#theDegrees = [2     2  2   4 3  4  4  3         2  2     4  2]                                                # [2 2 2 2 2 2  2   2   2 2  2  3  3    2    2 2 2 2 2  2   ] 
            # Intg S eps_0 r c qc ap sin_theta dA dtheta w x_0 # W Q E S P q q_e x_0 ap m m_e r r_0 lambda a b w e c eps_0 

candidate_mons = [
    #mons_of_max_degree_and_unit_overall(params, deg, deg_overall, [])
    all_monomials_up_to_max_deg_overall_and_individual(x, deg, deg_overall, theDegrees)
    for ai=axioms
]
@show size.(candidate_mons)

# model = Model(Mosek.Optimizer)
model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "TimeLimit", 600.0)

mons_q = mons_of_max_degree_and_unit_overall([ P qcs rs ws x_0s ], deg, deg_overall, []) #  only include variables expected to appear in the final theorem 
coeff_q =   @variable(model, [1:size(mons_q,1)], base_name="q")

q = sum(ci .* mi for (ci, mi)=zip(coeff_q, mons_q)) # Zip pairs things without needing a ref index, e.g., zip([1, 2, 3], [4,5,6])=((1,4), (2,5), (3,6))
coeff_αs = [
    @variable(model, [1:size(X,1)], base_name="α$i")
    for (i,X)=enumerate(candidate_mons)
    ]
@show size.(coeff_αs)
αs = [sum(ci .* mi) for (ci, mi)=zip(coeff_αs, candidate_mons)]

residual = q - sum(αᵢ * aᵢ for (αᵢ, aᵢ)=zip(αs,axioms));
eqs = coefficients(residual)

# Ensure that the sum of the coefficients on the terms involving m isn't zero, in order that m is part of expression
@constraint model sum(coeff_q[degree.(mons_q, [P ]).>0]) == 4.0 # insist that some variables will appear
#@constraint model sum(coeff_q[degree.(mons_q,[E_c]).>0]) == 0.0 # insist that some variables will not appear
@constraint model eqs .== 0
@objective model Max 0

optimize!(model)
@show termination_status(model)

size.(candidate_mons) = [(21539,), (21539,), (21539,), (21539,), (21539,)]
Set parameter Username
Academic license - for non-commercial use only - expires 2024-12-04
Set parameter TimeLimit to value 600
size.(coeff_αs) = [(21539,), (21539,), (21539,), (21539,), (21539,)]
Set parameter TimeLimit to value 600
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i9-10885H CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 125796 rows, 108698 columns and 217140 nonzeros
Model fingerprint: 0xa6f23944
Coefficient statistics:
  Matrix range     [5e-01, 6e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 4e+00]
Presolve removed 125796 rows and 108698 columns
Presolve time: 0.10s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0  

OPTIMAL::TerminationStatusCode = 1

In [11]:
value_poly = p -> sum(value.(coefficients(p)).* monomials(p))

#71 (generic function with 1 method)

In [12]:
value_q = value_poly(q)
value_q

4.0Prsws - 16.755160819145562rsqcsws³x_0s

In [14]:
@show value_αs = value_poly.(αs)

value_αs = value_poly.(αs) = Polynomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}, Float64}[3.0sin_theta³dAdthetaws, 4.0qcsapssin_theta²ws, 25.132741228718345rsqcsapsws - 3.0SrsdAws + 3.0qcsapssin_theta²dAws, 4.0rsws, 33.510321638291124rsqcsws]


5-element Vector{Polynomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}, Float64}}:
 3.0sin_theta³dAdthetaws
 4.0qcsapssin_theta²ws
 25.132741228718345rsqcsapsws - 3.0SrsdAws + 3.0qcsapssin_theta²dAws
 4.0rsws
 33.510321638291124rsqcsws